
# Scientific Image Forgery — **Submission Notebook (Inference Only)**

This notebook **loads a trained model** (TorchScript) and **generates `submission.csv`** for the competition.

- No training here — pure inference.  
- Follows the competition rule:  
  - Output `"authentic"` if no forged region is detected.  
  - Otherwise, output **RLE-encoded instance masks** using the **official encoding** (semicolons between instances).


## 1) Environment Report

In [ ]:

import sys, platform, torch, numpy as np, pandas as pd
print("Python:", sys.version.split()[0])
print("OS:", platform.platform())
print("Torch:", torch.__version__)
print("CUDA available:", torch.cuda.is_available())


## 2) Paths & Parameters

In [ ]:

from pathlib import Path
import os

# =========================
# EDIT THESE TWO DIRECTORIES
# =========================
COMP_DIR   = "/kaggle/input/recodai-luc-scientific-image-forgery-detection"
TEST_DIR   = f"{COMP_DIR}/test_images"

# Model artifact directory (where you uploaded your .pt)
MODEL_DIR  = "/kaggle/input/scientific-image-forgery-detection/pytorch/default/7" 
MODEL_FILE = "model_deeplabv3_fold0_ts.pt"               # TorchScript file saved earlier

# Output
OUT_DIR = "/kaggle/working"
os.makedirs(OUT_DIR, exist_ok=True)
SUB_PATH = f"{OUT_DIR}/submission.csv"

# Inference params
IMAGE_SIZE  = 512
THRESHOLD   = 0.5
MIN_AREA    = 64
USE_TTA     = True

DEVICE = "cuda" if torch.cuda.is_available() else "cpu"

print("TEST_DIR :", TEST_DIR)
print("MODEL_DIR:", MODEL_DIR)
print("MODEL_FILE:", MODEL_FILE)
print("Saving to:", SUB_PATH)


## 3) Utilities — RLE, Preprocessing, Post-processing

In [ ]:

import json
import numpy as np
import cv2
from PIL import Image
import torch

# --- Official RLE encode from the competition metric (user-provided) ---
try:
    import numba
    from numba import njit
except Exception as e:
    numba = None
    def njit(*args, **kwargs):
        def deco(f): return f
        return deco

@njit
def _rle_encode_jit(x: np.ndarray, fg_val: int = 1) -> list:
    dots = np.where(x.T.flatten() == fg_val)[0]
    run_lengths = []
    prev = -2
    for b in dots:
        if b > prev + 1:
            run_lengths.extend((b + 1, 0))
        run_lengths[-1] += 1
        prev = b
    return run_lengths

def rle_encode(masks: list[np.ndarray], fg_val: int = 1) -> str:
    return ';'.join([json.dumps(_rle_encode_jit(m.astype(np.uint8), fg_val)) for m in masks])


def load_rgb(path: str) -> Image.Image:
    return Image.open(path).convert("RGB")

def preprocess_pil(img: Image.Image, size: int) -> torch.Tensor:
    img_r = img.resize((size, size), resample=Image.BILINEAR)
    x = torch.from_numpy(np.array(img_r, dtype=np.float32) / 255.0).permute(2,0,1).unsqueeze(0)
    return x

def predict_prob(model, img: Image.Image, size: int, tta: bool = True) -> np.ndarray:
    x = preprocess_pil(img, size).to(DEVICE)
    with torch.no_grad():
        logits = model(x)  # TorchScript logits [B,1,H,W]
        prob = torch.sigmoid(logits)[0,0]
        if tta:
            xh = torch.flip(x, dims=[3])
            ph = torch.sigmoid(model(xh))[0,0]; ph = torch.flip(ph, dims=[1])
            xv = torch.flip(x, dims=[2])
            pv = torch.sigmoid(model(xv))[0,0]; pv = torch.flip(pv, dims=[0])
            prob = (prob + ph + pv) / 3.0
        return prob.detach().cpu().numpy()

def prob_to_instances(prob: np.ndarray, thr: float = 0.5, min_area: int = 64) -> list[np.ndarray]:
    mask = (prob > thr).astype(np.uint8)
    if mask.sum() < min_area:
        return []
    num, lbl, stats, _ = cv2.connectedComponentsWithStats(mask, connectivity=8)
    insts = []
    for i in range(1, num):
        area = int(stats[i, cv2.CC_STAT_AREA])
        if area >= min_area:
            inst = (lbl == i).astype(np.uint8)
            insts.append(inst)
    return insts


## 4) Load TorchScript Model

In [ ]:

import torch, os
ts_path = str(Path(MODEL_DIR) / MODEL_FILE)
assert os.path.exists(ts_path), f"Model file not found: {ts_path}"
model = torch.jit.load(ts_path, map_location=DEVICE).eval()
try:
    for p in model.parameters():
        p.requires_grad_(False)
except Exception:
    pass
print("Loaded TorchScript model from:", ts_path)


## 5) Run Inference on Test Set and Build `submission.csv`

In [ ]:
# 🧪 Inference + Submission (orig + H/V flip TTA merge) with low-confidence "authentic" gate
import os, glob, json
from pathlib import Path
import pandas as pd
import numpy as np
from PIL import ImageOps
import cv2

# --- Sanity checks for required globals from previous cells ---
needed_syms = [
    "load_rgb", "predict_prob", "prob_to_instances", "rle_encode",
    "IMAGE_SIZE", "USE_TTA", "THRESHOLD", "MIN_AREA",
    "TEST_DIR", "COMP_DIR", "model"
]
for _n in needed_syms:
    assert _n in globals(), f"Missing `{_n}`. Run the earlier setup/inference utility cells."

# Output path (define if not set)
if "SUB_PATH" not in globals():
    OUT_DIR = "/kaggle/working"
    os.makedirs(OUT_DIR, exist_ok=True)
    SUB_PATH = str(Path(OUT_DIR) / "submission.csv")

# ---- TTA + merge helpers ----
def prob_tta_merged(pil_img, size, tta):
    """Return merged prob map (resized grid, e.g., IMAGE_SIZE×IMAGE_SIZE) using mean(orig, H, V)."""
    # original
    p_orig = predict_prob(model, pil_img, size=size, tta=tta)  # (S,S) float in [0,1]

    # horizontal flip (mirror) -> unflip back
    p_h = predict_prob(model, ImageOps.mirror(pil_img), size=size, tta=tta)
    p_h = np.fliplr(p_h)

    # vertical flip -> unflip back
    p_v = predict_prob(model, ImageOps.flip(pil_img), size=size, tta=tta)
    p_v = np.flipud(p_v)

    # merge: mean (switch to np.maximum.reduce([...]) for higher recall if needed)
    p_m = (p_orig + p_h + p_v) / 3.0
    return np.clip(p_m, 0.0, 1.0), p_orig, p_h, p_v

# ---- Low-confidence "authentic" gate (ONLY for deciding authentic vs forged) ----
# If both "max prob" and "coverage above a low viz threshold" are tiny, force authentic.
LOW_CONF_MAX_PROB   = 0.06       # if the whole map is below ~6% prob → suspiciously low
LOW_VIZ_THR         = 0.04       # visualize/coverage threshold for "is there anything at all?"
LOW_CONF_MIN_PIXELS = 256        # at resized grid (IMAGE_SIZE×IMAGE_SIZE)

def is_low_confidence(prob_resized):
    """Return True if probabilities look too weak/rare to trust."""
    if float(prob_resized.max()) >= LOW_CONF_MAX_PROB:
        return False
    # Otherwise also check coverage at a small threshold (avoid 1-2 speckles)
    cover = int((prob_resized >= LOW_VIZ_THR).sum())
    return cover < LOW_CONF_MIN_PIXELS

# --- Collect test images ---
test_paths = sorted(glob.glob(str(Path(TEST_DIR) / "*")))
print("Test images:", len(test_paths))

rows = []

# If there are test images, run inference
if len(test_paths) > 0:
    for p in test_paths:
        case_id = Path(p).stem  # keep as string; we'll align types later
        img = load_rgb(p)

        # 1) Get merged probability at resized grid (consistent with your pipeline)
        prob_resized, p_o, p_h, p_v = prob_tta_merged(img, size=IMAGE_SIZE, tta=USE_TTA)

        # 2) Low-confidence gate: if too low → authentic
        if is_low_confidence(prob_resized):
            annot = "authentic"
        else:
            # 3) Normal instance extraction at your standard threshold
            instances = prob_to_instances(prob_resized, thr=THRESHOLD, min_area=MIN_AREA)
            annot = "authentic" if len(instances) == 0 else rle_encode(instances)

        rows.append({"case_id": case_id, "annotation": annot})

# Build dataframe (may be empty if no test files found)
sub = pd.DataFrame(rows, columns=["case_id", "annotation"])

# --- Align with sample_submission order & types (competition rule friendly) ---
ss_path = str(Path(COMP_DIR) / "sample_submission.csv")
if os.path.exists(ss_path):
    ss = pd.read_csv(ss_path)
    # Make merge key the same dtype on both sides (strings)
    ss["case_id"] = ss["case_id"].astype(str)
    if not sub.empty:
        sub["case_id"] = sub["case_id"].astype(str)
        sub = ss[["case_id"]].merge(sub, on="case_id", how="left")
        sub["annotation"] = sub["annotation"].fillna("authentic")
    else:
        # No predictions gathered (e.g., no test images visible) -> default to authentic
        sub = ss[["case_id"]].copy()
        sub["case_id"] = sub["case_id"].astype(str)
        sub["annotation"] = "authentic"
else:
    # No sample file -> ensure proper dtypes
    if not sub.empty:
        sub["case_id"] = sub["case_id"].astype(str)
    else:
        # Nothing to write and no sample to align to
        print("Warning: No test images and no sample_submission.csv found.")
        sub = pd.DataFrame(columns=["case_id", "annotation"])

# --- Save submission ---
sub.to_csv(SUB_PATH, index=False)
print("✅ Wrote submission:", SUB_PATH)
display(sub.head(10))

In [ ]:
# =====================
# Single-image visualization (orig + H/V flips) — self-contained version
# =====================
import numpy as np
import matplotlib.pyplot as plt
import cv2
from PIL import ImageOps
from pathlib import Path

if len(test_paths) == 1:
    p = test_paths[0]

    # --- Viz parameters ---
    VIS_THR  = 0.02
    CMAP_MAX = 0.05

    # --- local helpers ---
    def overlay_alpha(img_rgb, mask01, color=(255,0,0), alpha=0.35):
        if mask01.shape[:2] != img_rgb.shape[:2]:
            mask01 = cv2.resize(mask01, (img_rgb.shape[1], img_rgb.shape[0]), interpolation=cv2.INTER_NEAREST)
        if img_rgb.ndim == 2:
            base = cv2.cvtColor(img_rgb, cv2.COLOR_GRAY2BGR)
        elif img_rgb.shape[2] == 4:
            base = cv2.cvtColor(img_rgb, cv2.COLOR_RGBA2BGR)
        else:
            base = img_rgb.copy()
        if mask01.sum() == 0:
            return base
        overlay = base.copy()
        overlay[mask01 > 0] = (0.65*base[mask01 > 0] + 0.35*np.array(color)).astype(np.uint8)
        return overlay

    def draw_outline(img_rgb, mask01, color=(0,255,0), thickness=2):
        if mask01.shape[:2] != img_rgb.shape[:2]:
            mask01 = cv2.resize(mask01, (img_rgb.shape[1], img_rgb.shape[0]), interpolation=cv2.INTER_NEAREST)
        if img_rgb.ndim == 2:
            vis = cv2.cvtColor(img_rgb, cv2.COLOR_GRAY2BGR)
        elif img_rgb.shape[2] == 4:
            vis = cv2.cvtColor(img_rgb, cv2.COLOR_RGBA2BGR)
        else:
            vis = img_rgb.copy()
        if mask01.sum() == 0:
            return vis
        cnts, _ = cv2.findContours(mask01.astype(np.uint8), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cv2.drawContours(vis, cnts, -1, color, thickness, lineType=cv2.LINE_AA)
        return vis

    def to_orig_size(prob_resized, W, H):
        return cv2.resize(prob_resized, (W, H), interpolation=cv2.INTER_LINEAR)

    def bin_mask(prob_map, thr):
        return (prob_map >= thr).astype(np.uint8)

    # --- load image ---
    img_pil = load_rgb(p)
    img_np  = np.array(img_pil)
    H, W    = img_np.shape[:2]

    # --- compute TTA probabilities ---
    prob_orig = predict_prob(model, img_pil, size=IMAGE_SIZE, tta=USE_TTA)
    prob_h    = np.fliplr(predict_prob(model, ImageOps.mirror(img_pil), size=IMAGE_SIZE, tta=USE_TTA))
    prob_v    = np.flipud(predict_prob(model, ImageOps.flip(img_pil), size=IMAGE_SIZE, tta=USE_TTA))
    prob_m    = np.clip((prob_orig + prob_h + prob_v) / 3.0, 0.0, 1.0)

    # --- resize to original ---
    p_orig = to_orig_size(prob_orig, W, H)
    p_h    = to_orig_size(prob_h, W, H)
    p_v    = to_orig_size(prob_v, W, H)
    p_m    = to_orig_size(prob_m, W, H)

    # --- low-threshold masks ---
    m_orig = bin_mask(p_orig, VIS_THR)
    m_h    = bin_mask(p_h, VIS_THR)
    m_v    = bin_mask(p_v, VIS_THR)
    m_m    = bin_mask(p_m, VIS_THR)

    # --- quick stats ---
    def pstats(name, p):
        vals = p.ravel()
        print(f"{name:>6} | min={vals.min():.4f} mean={vals.mean():.4f} max={vals.max():.4f} "
              f"p99={np.quantile(vals,0.99):.4f} p99.5={np.quantile(vals,0.995):.4f}")

    print(f"File: {Path(p).name} | {W}×{H} | VIS_THR={VIS_THR:.3f} | cmap vmax={CMAP_MAX:.3f}")
    pstats("orig",  p_orig)
    pstats("hflip", p_h)
    pstats("vflip", p_v)
    pstats("merge", p_m)

    # --- visuals ---
    fig = plt.figure(figsize=(18, 12))
    ax1 = plt.subplot(3,4,1); im1=ax1.imshow(p_orig,cmap='magma',vmin=0,vmax=CMAP_MAX); ax1.set_title('Prob (orig)'); ax1.axis('off'); plt.colorbar(im1,ax=ax1,fraction=0.046,pad=0.02)
    ax2 = plt.subplot(3,4,2); im2=ax2.imshow(p_h,   cmap='magma',vmin=0,vmax=CMAP_MAX); ax2.set_title('Prob (H→unflip)'); ax2.axis('off'); plt.colorbar(im2,ax=ax2,fraction=0.046,pad=0.02)
    ax3 = plt.subplot(3,4,3); im3=ax3.imshow(p_v,   cmap='magma',vmin=0,vmax=CMAP_MAX); ax3.set_title('Prob (V→unflip)'); ax3.axis('off'); plt.colorbar(im3,ax=ax3,fraction=0.046,pad=0.02)
    ax4 = plt.subplot(3,4,4); im4=ax4.imshow(p_m,   cmap='magma',vmin=0,vmax=CMAP_MAX); ax4.set_title('Prob (merged)'); ax4.axis('off'); plt.colorbar(im4,ax=ax4,fraction=0.046,pad=0.02)

    ax5 = plt.subplot(3,4,5); ax5.imshow(m_orig,cmap='gray'); ax5.set_title(f'Binary ≥{VIS_THR:.3f} (orig)'); ax5.axis('off')
    ax6 = plt.subplot(3,4,6); ax6.imshow(m_h,   cmap='gray'); ax6.set_title(f'Binary ≥{VIS_THR:.3f} (H)'); ax6.axis('off')
    ax7 = plt.subplot(3,4,7); ax7.imshow(m_v,   cmap='gray'); ax7.set_title(f'Binary ≥{VIS_THR:.3f} (V)'); ax7.axis('off')
    ax8 = plt.subplot(3,4,8); ax8.imshow(m_m,   cmap='gray'); ax8.set_title(f'Binary ≥{VIS_THR:.3f} (merged)'); ax8.axis('off')

    ov  = overlay_alpha(img_np, m_m, (255,0,0), 0.35)
    out = draw_outline(img_np, m_m, (0,255,0), thickness=max(2, int(0.003*max(H,W))))

    ax9  = plt.subplot(3,4,9);  ax9.imshow(ov);  ax9.set_title('Overlay (merged)'); ax9.axis('off')
    ax10 = plt.subplot(3,4,10); ax10.imshow(out); ax10.set_title('Outline (merged)'); ax10.axis('off')

    ax11 = plt.subplot(3,4,11)
    if m_m.sum()>0:
        ys,xs=np.where(m_m>0); y0,y1=max(0,ys.min()-20),min(H,ys.max()+20); x0,x1=max(0,xs.min()-20),min(W,xs.max()+20)
        ax11.imshow(out[y0:y1,x0:x1]); ax11.set_title('Zoom (merged)'); ax11.axis('off')
    else:
        ax11.imshow(out); ax11.set_title('Zoom (merged) - empty'); ax11.axis('off')

    ax12=plt.subplot(3,4,12)
    vals=p_m.ravel(); ax12.hist(vals,bins=100,range=(0,CMAP_MAX))
    ax12.axvline(VIS_THR,color='r',ls='--',label=f'VIS_THR={VIS_THR:.3f}')
    ax12.set_title('Histogram (merged probs)'); ax12.legend()

    plt.tight_layout(); plt.show()
else:
    print("⚠️ This cell is for single-image visual only (found", len(test_paths), ")")


# Note

Here, the forged cells are not detected, but we can still see that the model try to predicting it with a prob of 0.04 at the good position, so it's a great start!